# Recuperando dados de API e salvando em banco de dados

By: [Gizélly N.S.](https://www.linkedin.com/in/gizellyns/)   

Utilizando o endpoint de scores para buscar determinadas informações do sport soccer_brazil_campeonato na resposta da API e salvar
numa tabela chamada partidas_brasileirao_serie_a_2023 contendo as seguintes colunas apenas os resultados do ano de 2023:
* data_hora_partida 
* data_partida
* time_casa
* time_fora
* gols_time_casa
* gols_time_fora


In [1]:
import requests
import pandas as pd
import json
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, TimestampType, FloatType, DoubleType

from modules.write_on_local_db import write_on_local_db
from modules.import_conn_properties import import_conn_properties  
from modules.get_from_db import get_from_db

spark = SparkSession \
    .builder \
    .appName('03_api') \
    .config('spark.driver.extraClassPath', 'postgresql-42.2.10.jar') \
    .getOrCreate()

In [2]:
!pip install psycopg2-binary

In [3]:


url = "https://odds.p.rapidapi.com/v4/sports/soccer_brazil_campeonato/scores"
querystring = {"daysFrom": "3"}
headers = {
    "X-RapidAPI-Key": "7ce70cd8fbmsh16dd692b2bc06c3p186c76jsn4dfac271adfb",
    "X-RapidAPI-Host": "odds.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
content = response.json()
    
df_normalized = pd.json_normalize(content)
df_normalized["data_hora_partida"] = pd.to_datetime(df_normalized['commence_time'])
df_normalized["data_partida"] = df_normalized['data_hora_partida'].dt.date
df_normalized["ano"] = df_normalized['data_hora_partida'].dt.year

df_filtered = df_normalized[df_normalized["ano"] == 2023]
df_filtered = df_filtered.rename(columns = {"home_team":"time_casa", "away_team":"time_fora"})

df_filtered['scores'] = df_filtered['scores'].fillna("")
df_filtered = pd.concat([df_filtered.drop(['scores'], axis=1), df_filtered['scores'].apply(pd.Series)], axis=1)


time_casa = pd.json_normalize(df_filtered[0])
time_casa = time_casa.rename(columns={"score":"gols_time_casa"})
time_casa = time_casa.loc[:,"gols_time_casa"]

time_fora = pd.json_normalize(df_filtered[1])
time_fora = time_fora.rename(columns=({"score":"gols_time_fora"}))
time_fora = time_fora.loc[:,"gols_time_fora"]


df_filtered = df_filtered.loc[:,["data_hora_partida", "data_partida", "time_casa", "time_fora"]]



In [4]:
df = df_filtered.join(time_casa)
consolidado = df.join(time_fora)

### Criando tabela no banco de dados

In [5]:
%run -i './sql/create_table_partidas_brasileirao_serie_a_2023.py'

Tabela results criada com sucesso.


### Salvando em banco de dados

In [6]:
partidas_schema = StructType(
        [
            StructField("data_hora_partida", TimestampType(), True),
            StructField("data_partida", DateType(), True),
            StructField("time_casa", StringType(), True),
            StructField("time_fora", StringType(), True),
            StructField("gols_time_casa", StringType(), True),
            StructField("gols_time_fora", StringType(), True)
        ]
    )

consolidado = spark.createDataFrame(consolidado, schema=partidas_schema)

bra_properties = import_conn_properties('./conn_properties/brasileirao_serie_a_2023_properties.txt')

write_on_local_db(consolidado, bra_properties)

'Sucess'

### Lendo os dados inseridos no banco

In [7]:
get_from_db(spark, bra_properties, "brasileirao_serie_a_2023")

spark.sql("SELECT  * FROM brasileirao_serie_a_2023").show()

+-------------------+------------+-------------------+----------------+--------------+--------------+
|  data_hora_partida|data_partida|          time_casa|       time_fora|gols_time_casa|gols_time_fora|
+-------------------+------------+-------------------+----------------+--------------+--------------+
|2023-07-22 21:31:16|  2023-07-22|              Bahia|     Corinthians|             0|             0|
|2023-07-23 19:00:00|  2023-07-23|      Bragantino-SP|   Internacional|           NaN|           NaN|
|2023-07-23 00:00:00|  2023-07-23|             Gremio|Atletico Mineiro|           NaN|           NaN|
|2023-07-23 19:00:00|  2023-07-23|           Cruzeiro|           Goiás|           NaN|           NaN|
|2023-07-29 19:00:00|  2023-07-29|      Internacional|          Cuiabá|           NaN|           NaN|
|2023-07-23 19:00:00|  2023-07-23|             Santos|        Botafogo|           NaN|           NaN|
|2023-07-30 19:00:00|  2023-07-30|           Botafogo|        Coritiba|           